<a href="https://colab.research.google.com/github/saivenkatreddy29/Building-LLM-from-Scratch/blob/main/Chapter_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.2 MB/s eta 0:00:00


In [ ]:
import torch
from importlib.metadata import version

print('torch version', version('torch'))
print('tiktoken version',version('tiktoken'))

torch version 2.5.1+cu121
tiktoken version 0.8.0


In [ ]:
import os
import requests
path = "the-verdict.txt"
url =  ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
if os.path.exists(path):
  print("The file already exist")
else:
  print("The file doesn't exist downloading.....")
  response = requests.get(url)
  if response.status_code == 200:
    print('Download completed')
    with open(path,'wb') as f:
      f.write(response.content)
  else:
    print('Download Failed')


The file doesn't exist downloading.....
Download completed


In [ ]:
import re
text =  "Hello, world. This, is a test."
result = re.split(r'\s',text)
print(result)

['Hello,', 'world.', 'This,', 'is', 'a', 'test.']


In [ ]:
result = re.split(r'([,.]|\s)',text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [ ]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [ ]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)',text)
result = [item for item in result if item.strip()]

In [ ]:
result

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']

In [ ]:
with open(path, 'r',  encoding="utf-8") as f:
  raw_text = f.read()

In [ ]:
raw_text[:10]

'I HAD alwa'

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

In [ ]:
print(preprocessed[:20])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was']


In [ ]:
len(preprocessed)

4690

In [ ]:
preprocessed = sorted(set(preprocessed))
preprocessed[:20]

['!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '--',
 '.',
 ':',
 ';',
 '?',
 'A',
 'Ah',
 'Among',
 'And',
 'Are',
 'Arrt',
 'As',
 'At',
 'Be']

In [ ]:
len(preprocessed)

1130

In [ ]:
word_to_in = {}
in_to_word = {}

for i,word in enumerate(preprocessed):
  word_to_in[word] = i
  in_to_word[i] = word

In [ ]:
for word,key in word_to_in.items():
  print(f'word:{word},key:{key}')
  if key>=20:
    break

word:!,key:0
word:",key:1
word:',key:2
word:(,key:3
word:),key:4
word:,,key:5
word:--,key:6
word:.,key:7
word::,key:8
word:;,key:9
word:?,key:10
word:A,key:11
word:Ah,key:12
word:Among,key:13
word:And,key:14
word:Are,key:15
word:Arrt,key:16
word:As,key:17
word:At,key:18
word:Be,key:19
word:Begin,key:20


In [ ]:
for key,word in in_to_word.items():
  print(f'key:{key} word:{word}')
  if key>=20:
    break

key:0 word:!
key:1 word:"
key:2 word:'
key:3 word:(
key:4 word:)
key:5 word:,
key:6 word:--
key:7 word:.
key:8 word::
key:9 word:;
key:10 word:?
key:11 word:A
key:12 word:Ah
key:13 word:Among
key:14 word:And
key:15 word:Are
key:16 word:Arrt
key:17 word:As
key:18 word:At
key:19 word:Be
key:20 word:Begin


In [ ]:
class SimpleTokenizerv1():
  def __init__(self,vocabulary):

    self.word_to_key = {}
    self.key_to_word = {}
    for i,word in enumerate(preprocessed):
      self.word_to_key[word] = i
      self.key_to_word[i] = word


  def encode(self,text):
    pro_text = re.split(r'([,.:;?_!"()\']|--|\s)',text)
    pro_text = [item.strip() for item in pro_text if item.strip()]
    encoded = []
    for word in pro_text:
      encoded.append(self.word_to_key[word])
    return encoded

  def decode(self,encoded):
    ans = []
    # print(self.key_to_word)
    for key in encoded:
      ans.append(self.key_to_word[key])
    text = ' '.join(ans)
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text




In [ ]:
#preprocessed
text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""

In [ ]:
tokenizer = SimpleTokenizerv1(preprocessed)
encoded_text = tokenizer.encode(text)
print(encoded_text)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [ ]:
decoded_text = tokenizer.decode(encoded_text)
print(decoded_text)

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [ ]:
text = "Hello, do you like tea. Is this-- a test?"


Adding Special tokens to the simple tokenizer

In [ ]:
preprocessed.extend(["<|endoftext|>", "<|unk|>"])

In [ ]:
len(preprocessed)

1132

In [ ]:
preprocessed[-3:]

['yourself', '<|endoftext|>', '<|unk|>']

In [ ]:
class SimpleTokenizerv2:
  def __init__(self, vocabulary):
    self.word_to_int = {}
    self.int_to_word = {}

    for key,word in enumerate(vocabulary):
      self.word_to_int[word] = key
      self.int_to_word[key] = word

  def encode(self, query):
    # print(query)
    ans = []
    text = re.split(r'([,.:;?_!"()\']|--|\s)',query)
    text = [result.strip() for result in text if result.strip()]
    # print(f'text:{text}')
    for word in text:
      if word in self.word_to_int:
        ans.append(self.word_to_int[word])
      else:
        ans.append(self.word_to_int["<|unk|>"])
    return ans
  def decode(self,code):
    ans = []
    for token in code:
      ans.append(self.int_to_word[token])
    text = ' '.join(ans)
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text





In [ ]:
tokenizer = SimpleTokenizerv2(preprocessed)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = ' <|endoftext|> '.join((text1,text2))
print(text)



Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [ ]:
encoded_text = tokenizer.encode(text)
print(encoded_text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [ ]:
decoded_text = tokenizer.decode(encoded_text)
print(decoded_text)

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


# Byte Pair Encoding

In [ ]:
import importlib
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')

In [ ]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text,
                            allowed_special = {"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [ ]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


# Data Sampling with sliding window

In [ ]:
with open(path,'r',encoding = 'utf-8') as f:
  raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [ ]:
enc_text[:10]

[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138]

In [ ]:
x = enc_text[:4]
y = enc_text[1:4+1]
print(f'input: {x}')
print(f'target output: {y}')

input: [40, 367, 2885, 1464]
target output: [367, 2885, 1464, 1807]


In [ ]:
context_size = 4
for i in range(1,context_size):
  context = enc_text[:i]
  label = enc_text[:i+1]
  print(f'context: {context}')
  print(f'label: {label}')

context: [40]
label: [40, 367]
context: [40, 367]
label: [40, 367, 2885]
context: [40, 367, 2885]
label: [40, 367, 2885, 1464]


# Creating a dataset

In [ ]:
from torch.utils.data import Dataset,DataLoader
class GPTDatasetv1(Dataset):
  def __init__(self,text,tokenizer,
               context_len, stride):
    self.text = tokenizer.encode(text,
                                 allowed_special = {"<|endoftext|>"})
    self.inputs = []

    self.labels = []
    self.len = len(self.text)
    for i in range(1,self.len - context_len,stride):
      self.input = self.text[i:i+context_len]
      self.label = self.text[i+1:i+context_len+1]
      self.inputs.append(torch.tensor(self.input))
      self.labels.append(torch.tensor(self.label))
  def __len__(self):
    return len(self.inputs)

  def __getitem__(self,idx):
    return self.inputs[idx], self.labels[idx]





In [ ]:
enc = GPTDatasetv1(raw_text,
                   tokenizer,
                   context_len = 4,
                   stride = 1)

In [ ]:
enc.__len__()

5140

In [ ]:
enc.__getitem__(2)

(tensor([1464, 1807, 3619,  402]), tensor([1807, 3619,  402,  271]))

In [ ]:
def create_dataloader(text,
                      batch_size = 4,
                      context_len = 256,
                      stride = 128,
                      shuffle = True,
                      drop_last = True,
                      num_workers = 0
                      ):
  tokenizer = tiktoken.get_encoding('gpt2')
  dataset = GPTDatasetv1(text,
                   tokenizer,
                   context_len = context_len,
                   stride = stride)
  dataloader = DataLoader(
      dataset = dataset,
      batch_size = batch_size,
      shuffle = shuffle,
      drop_last = drop_last,
      num_workers = 0

  )
  return dataloader

In [ ]:
dataloader = create_dataloader(text = raw_text,context_len = 4,
                      stride = 1,shuffle = False)

In [ ]:
dataloader

In [ ]:
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)


[tensor([[ 367, 2885, 1464, 1807],
        [2885, 1464, 1807, 3619],
        [1464, 1807, 3619,  402],
        [1807, 3619,  402,  271]]), tensor([[ 2885,  1464,  1807,  3619],
        [ 1464,  1807,  3619,   402],
        [ 1807,  3619,   402,   271],
        [ 3619,   402,   271, 10899]])]


In [ ]:
dataloader = create_dataloader(raw_text,
                               batch_size = 8,
                               context_len = 4,
                               stride = 4,
                               shuffle = False)

data_iter = iter(dataloader)
print(next(data_iter))

[tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]]), tensor([[ 2885,  1464,  1807,  3619],
        [  402,   271, 10899,  2138],
        [  257,  7026, 15632,   438],
        [ 2016,   257,   922,  5891],
        [ 1576,   438,   568,   340],
        [  373,   645,  1049,  5975],
        [  284,   502,   284,  3285],
        [  326,    11,   287,   262]])]


# Creating Token Embeddings

In [ ]:
input_ids = torch.tensor([2,3,5,1])

In [ ]:
from torch import nn
vocab_size = 6
embedding_dim = 3


torch.manual_seed(123)
embedding_layer = nn.Embedding(vocab_size,embedding_dim)

In [ ]:
input_embeddings = embedding_layer(input_ids)
print(input_embeddings)

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [ ]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


Encoding Word Positions

In [ ]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = nn.Embedding(vocab_size,output_dim)

In [ ]:
max_length = 4
dataloader = create_dataloader(raw_text,
                               batch_size = 8,
                               context_len = 4,
                               stride = 4,
                               shuffle = False)
data_iter = iter(dataloader)
inputs,targets = next(data_iter)

In [ ]:
print(f"Inputs: {inputs}\n")
print(f"Outputs :{targets}\n")

Inputs: tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])

Outputs :tensor([[ 2885,  1464,  1807,  3619],
        [  402,   271, 10899,  2138],
        [  257,  7026, 15632,   438],
        [ 2016,   257,   922,  5891],
        [ 1576,   438,   568,   340],
        [  373,   645,  1049,  5975],
        [  284,   502,   284,  3285],
        [  326,    11,   287,   262]])



In [ ]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


Add positional embeddings

In [ ]:
context_size

4

In [ ]:
positional_embedding_layer = nn.Embedding(context_size,output_dim)

In [ ]:
posititonal_embeddings = positional_embedding_layer(torch.arange(context_size))
posititonal_embeddings.shape

torch.Size([4, 256])

In [ ]:
input_embeddings = token_embeddings+posititonal_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


In [ ]:
token_embeddings[2]

tensor([[ 0.8046,  0.5047,  1.2922,  ...,  1.4648,  0.4097,  0.3205],
        [ 0.0795, -1.7636,  0.5750,  ...,  2.1823,  1.8231, -0.3635],
        [ 0.4267, -0.0647,  0.5686,  ..., -0.5209,  1.3065,  0.8473],
        [ 0.4256, -0.2887, -1.1954,  ..., -1.1536,  1.8305, -0.1415]],
       grad_fn=<SelectBackward0>)

In [ ]:
input_embeddings[2]

tensor([[ 2.5421, -0.0572,  0.6619,  ...,  1.2371,  1.9845,  1.3550],
        [ 1.7217, -2.4837,  0.7812,  ...,  2.5940,  1.9729, -0.8263],
        [-0.0384, -0.8404,  1.1492,  ...,  0.9125,  0.8102,  1.7052],
        [-0.2498, -0.7516,  0.2369,  ..., -0.3397,  1.1217,  0.3412]],
       grad_fn=<SelectBackward0>)